import sys
!{sys.executable} -m pip install wordcloud-1.8.1-cp39-cp39-win_amd64.whl


#**Analisis Sentimen pada Permendikbud**



---



Nama Peserta:
- Ferdy Nanda Riza
- Shabrina Novaindah Dwiyamti
- Nalurisa Izma Mardiana

Universitas Host: Universitas Telkom

Kelas: TelU-04

Kelompok: 3


#Import Library yang dibutuhkan

In [ ]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from google.colab import files
import io

#Mengupload file yang berisi kode API yang telah didapat

Untuk mendapatkan kode API kita harus terdaftar sebagai developer dan mengisi berbagai pertanyaan yang diajukan.

In [ ]:
#Mengambil data
upload = files.upload()

Saving API.csv to API.csv


In [ ]:
#membaca data dari file csv
log = pd.read_csv(io.BytesIO(upload['API.csv']))

In [ ]:
#API Twitter
consumerKey = log['APIkey'][0]
consumerSecret = log['APIKeysecret'][0]
accessToken = log['accestoken'][0]
accessTokenSecret = log['accestokensecret'][0]

In [ ]:
#Membuat objek autentikasi
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)

#Mengatur akses token
authenticate.set_access_token(accessToken, accessTokenSecret)

#Membuat objek API saat otentifikasi
api = tweepy.API(authenticate, wait_on_rate_limit=True)

#Menentukan topik apa yang akan dilakukan analisis

Setelah kode API didapatkan, maka langkah selanjutnya adalah melakukan filter untuk data yang akan dilakukan analisis. Kami menggunakan data berdasarkan kolom pencarian dengan kata Permendikbud, dengan 1000 tweet terakhir menggunakan bahasa Indonesia.

In [ ]:
#melakukan filter untuk data 
search_term = 'Permendikbud -filter:retweets'
tweet_amount = 1000
since_date = '2021-10-25'

In [ ]:
#membuat list
posts = list(tweepy.Cursor(api.search, q=search_term, lang='id', tweet_mode='extended').items(tweet_amount))

In [ ]:
import json #mengkonversi kamus python ke bentuk json string
from pandas.io.json import json_normalize

# function to convert _json to JSON
def jsonify_tweepy(tweepy_object):
    json_str = json.dumps(tweepy_object._json)
    return json.loads(json_str)


In [ ]:
# Call the function and unload each _json into follower_list
posts_list = [jsonify_tweepy(post) for post in posts]

# Convert followers_list to a pandas dataframe
df = json_normalize(posts_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


#Melihat data frame 5 terbawah

In [ ]:
#melihat data frame 5 terbawah
df.tail()

,created_at,id,id_str,full_text,truncated,display_text_range,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,geo,coordinates,place,contributors,is_quote_status,quoted_status_id,quoted_status_id_str,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,entities.hashtags,entities.symbols,entities.user_mentions,entities.urls,metadata.iso_language_code,metadata.result_type,user.id,user.id_str,user.name,user.screen_name,user.location,user.description,user.url,user.entities.description.urls,user.protected,...,quoted_status.user.profile_sidebar_fill_color,quoted_status.user.profile_text_color,quoted_status.user.profile_use_background_image,quoted_status.user.has_extended_profile,quoted_status.user.default_profile,quoted_status.user.default_profile_image,quoted_status.user.following,quoted_status.user.follow_request_sent,quoted_status.user.notifications,quoted_status.user.translator_type,quoted_status.user.withheld_in_countries,quoted_status.geo,quoted_status.coordinates,quoted_status.place,quoted_status.contributors,quoted_status.is_quote_status,quoted_status.retweet_count,quoted_status.favorite_count,quoted_status.favorited,quoted_status.retweeted,quoted_status.possibly_sensitive,quoted_status.lang,quoted_status.user.entities.url.urls,entities.media,extended_entities.media,user.entities.url.urls,place.id,place.url,place.place_type,place.name,place.full_name,place.country_code,place.country,place.contained_within,place.bounding_box.type,place.bounding_box.coordinates,quoted_status.entities.media,quoted_status.extended_entities.media,quoted_status.quoted_status_id,quoted_status.quoted_status_id_str
995,Thu Nov 18 00:05:19 +0000 2021,1461123316945588224,1461123316945588224,@muslimahnewscom Tolak permendikbud PPKS,False,"[17, 40]","<a href=""http://twitter.com/download/android"" ...",1.461116e+18,1461115796977979393,1.434670e+18,1434669800588931073,muslimahnewscom,None,None,NaN,None,False,NaN,NaN,0,0,False,False,NaN,in,[],[],"[{'screen_name': 'muslimahnewscom', 'name': 'M...",[],in,recent,1431286087096279040,1431286087096279040,Alice,Indomietelor021,,Sayang Indonesia #GakFollowPorno,None,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,Wed Nov 17 23:51:14 +0000 2021,1461119771080986625,1461119771080986625,Bagi perguruan tinggi yang tidak melaksanakan...,False,"[0, 243]","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,None,None,None,None,NaN,None,False,NaN,NaN,0,2,False,False,False,in,[],[],[],[],in,recent,1444992335234830339,1444992335234830339,Ummu,adhelummu,,Entah acun yg ke berapa\nSemoga bisa grow up d...,None,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 1461119759773143046, 'id_str': '146111...","[{'id': 1461119759773143046, 'id_str': '146111...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,Wed Nov 17 23:49:49 +0000 2021,1461119416888750083,1461119416888750083,perguruan tinggi yang tidak melaksanakan Perme...,False,"[0, 280]","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,None,None,None,None,NaN,None,False,NaN,NaN,0,1,False,False,False,in,[],[],[],[],in,recent,1448636632844832768,1448636632844832768,tamm2019,tamm20191,,"Tidak usah terlalu serius, yang penting seru",None,[],False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 1461119414514769920, 'id_str': '146111...","[{'id': 1461119414514769920, 'id_str': '146111...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,Wed Nov 17 23:38:20 +0000 2021,1461116523745996802,1461116523745996802,Sanksi Bagi Kampus yang Mengabaikan PPKS diang...,False,"[0, 274]","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,None,None,None,None,NaN,None,False,NaN,NaN,0,1,Fals

In [ ]:
#mengganti nama column full_text menjadi Twets lalu menampilkan 5 data teratas
tweet_df = df[['full_text']]
tweet_df = tweet_df.rename(columns={'full_text' : 'Tweets'})
tweet_df.head()

,Tweets
0,Permendikbud 30 adalah sebuah ikhtiar di tenga...
1,"Permendikbud-Ristek 30/2021, seolah-olah terse..."
2,Permendikbudristek No.30 Tahun 2021 tentang Pe...
3,Prof. Panut Mulyono menyampaikan bahwa kehadir...
4,Dalam Permendikbudristek No.30 Tahun 2021 tent...


#Membersihkan data

Terdapat beberapa kata dan simbol yang tidak dibutuhkan dalam analisis sentimental, maka untuk tweet yang mengandung kata dan simbol tersebut harus dihilangkan. Misalnya, menghilangkan simbol @ untuk nama pengguna Twitter, simbol # untuk tagar topik yang dibicarakan dan https untuk link berita online.

In [ ]:
#Clean text

#Membuat fungsi membersihkan tweet berupa kata atau simbol yang tidak dibutuhkan 
def cleanTxt(text) :
  text = re.sub(r'@[A-Za-z0-9_]+', '', text)
  text = re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', '', text)
  text = re.sub(r'#', '', text)
  text = re.sub(r'https?:\/\/\S+', '', text)
  text = re.sub(r'^\s+', '', text)
  return text

In [ ]:
gg = 'kira-kira @__fnr_999 - ggg @fr_77 '
#re.sub(r'(@[A-Za-z0-9_]+)|(\w+:\/\/\S+)', '', gg)
#re.sub(r'@[A-Za-z0-9_]+', '', gg)
re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', '', gg)

'kirakira fnr999  ggg 77'

In [ ]:
#menerapkan fungsi cleanTxt pada column Tweets
tweet_df['Tweets'] = tweet_df['Tweets'].apply(cleanTxt)
tweet_df

,Tweets
0,Permendikbud 30 adalah sebuah ikhtiar di tenga...
1,PermendikbudRistek 302021 seolaholah terselina...
2,Permendikbudristek No30 Tahun 2021 tentang Pen...
3,Prof Panut Mulyono menyampaikan bahwa kehadira...
4,Dalam Permendikbudristek No30 Tahun 2021 tenta...
...,...
995,Tolak permendikbud PPKS
996,Bagi perguruan tinggi yang tidak melaksanakan...
997,perguruan tinggi yang tidak melaksanakan Perme...
998,Sanksi Bagi Kampus yang Mengabaikan PPKS diang...


In [ ]:
#menampilkan 10 Tweets teratas
for tweet in tweet_df['Tweets'][0:10]:
  print(tweet)
  print('')

Permendikbud 30 adalah sebuah ikhtiar di tengah kedaruratan Kondisi rill begini Gada hubungan sama sekali dg seks bebas dan segala klaim sekitarnya Kak  lekas pulih dan terima kasih youre doing the right thing  

PermendikbudRistek 302021 seolaholah terselinap ada neraka yang sudah dipesan 

Permendikbudristek No30 Tahun 2021 tentang Pencegahan dan Penanganan Kekerasan Seksual yang terjadi di lingkungan Pendidikan Tinggi ini dapat kita lihat ada dua potret buram dengan munculnya keberadaan Permendikbud ini Simak selengkapnya di video mmc 

Prof Panut Mulyono menyampaikan bahwa kehadiran PermendikbudRistek No 30 tahun 2021 adalah bentuk kepastian hukum yang diberikan oleh pemerintah sebagai bentuk keresahan atas meningkatnya kasus kekerasan seksual di perguruan tinggi FRI permendikbud30 

Dalam Permendikbudristek No30 Tahun 2021 tentang Pencegahan dan Penanganan Kekerasan Seksual yang terjadi di lingkungan Pendidikan Tinggi ini dapat kita lihat ada dua potret buram dengan munculnya kebe

#Melihat tanggal dibuatnya 10 tweet teratas

In [ ]:
# menampilkan tanggal dibuatnya 10 Tweets teratas
df['created_at'].tail(10)

990    Thu Nov 18 00:13:58 +0000 2021
991    Thu Nov 18 00:13:15 +0000 2021
992    Thu Nov 18 00:10:57 +0000 2021
993    Thu Nov 18 00:08:10 +0000 2021
994    Thu Nov 18 00:07:05 +0000 2021
995    Thu Nov 18 00:05:19 +0000 2021
996    Wed Nov 17 23:51:14 +0000 2021
997    Wed Nov 17 23:49:49 +0000 2021
998    Wed Nov 17 23:38:20 +0000 2021
999    Wed Nov 17 23:37:02 +0000 2021
Name: created_at, dtype: object

#Melakukan import data dari 1000 tweet ke dalam file csv

In [ ]:
#import file data frame ke csv
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
#menyimpan data hasil import ke google drive
tweet_df.to_csv('tweet_ttg_permendikbud.csv', index=False)
!cp tweet_ttg_permendikbud.csv "drive/My Drive/Colab Notebooks/Tugas besar/"